### Combining GDP data with the jobs report

In [1]:
%matplotlib inline
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

qtrs = {1: 1, 2: 1, 3: 1, 4: 2, 5:2, 6:2, 7:3, 8:3, 9:3, 10:4, 11:4, 12:4}

In [2]:
aah = {}
epop = {}
cols = ['HRSACTT', 'LFS', 'BASICWGT', 'MONTH', 'PWSSWGT']
for year in range(1989, 2021):
    if year >= 1998:
        wgt = 'PWSSWGT'
    else:
        wgt = 'BASICWGT'
    df = pd.read_feather(cps_dir / f'cps{year}.ft', columns=cols)
    ah = (df.query('LFS == "Employed"')
            .groupby('MONTH').apply(lambda x: np.average(x.HRSACTT, weights=x.BASICWGT)))
    aah.update({f'{year}-{month}-01': value for month, value in list(zip(ah.index, ah.values))})
    
    ep = (df.query('LFS == "Employed"').groupby('MONTH')[wgt].sum() / 
          df.groupby('MONTH')[wgt].sum())
    epop.update({f'{year}-{month}-01': value for month, value in list(zip(ep.index, ep.values))})

In [3]:
sm = x13_arima_analysis(pd.Series(epop, index=pd.to_datetime(list(epop.keys()))))
epop_sa = sm.trend.resample('QS').mean()
sm = x13_arima_analysis(pd.Series(aah, index=pd.to_datetime(list(aah.keys()))))
aah_sa = sm.trend.resample('QS').mean()

/home/brian/miniconda3/lib/python3.8/site-packages/statsmodels/tsa/x13.py:188: X13Warning: NOTE: Unable to test LS2020.Mar due to regression matrix singularity.
       The effect of this outlier is already accounted for by other regressors 
       (usually user-specified or previously identified outliers).
  
          found in one or more of the estimated spectra.
  warn(errors, X13Warning)


In [4]:
gdp_code = ('T10106', ['A191RX'])
pop_code = ('T70100', ['B230RC'])
gdp = nipa_df(retrieve_table(gdp_code[0])['Data'], gdp_code[1]).sort_index()
pop = nipa_df(retrieve_table(pop_code[0])['Data'], pop_code[1]).sort_index()

df = pd.DataFrame()
df['epop'] = epop_sa
df['pop'] = pop['B230RC']
df['hours'] = aah_sa * 52
df['gdp'] = gdp['A191RX']
df['input'] = (df['pop'] * df['epop']) * df['hours']
df['gdpinp'] = df['gdp'] / df['input']


df['epop_c'] = ((df['pop'] * df['epop'].mean()) * df['hours']) * df['gdpinp']
df['pop_c'] = ((df['pop'].mean() * df['epop']) * df['hours']) * df['gdpinp']
df['hours_c'] = ((df['pop'] * df['epop']) * df['hours'].mean()) * df['gdpinp']

df_g = growth_rate(df)

df_g['pop_contr'] = df_g['gdp'] - df_g['pop_c']
df_g['epop_contr'] = df_g['gdp'] - df_g['epop_c']
df_g['hours_contr'] = df_g['gdp'] - df_g['hours_c']
df_g['prod'] = df_g['gdp'] - df_g['pop_contr'] - df_g['epop_contr'] - df_g['hours_contr']

result = df_g[['pop_contr', 'epop_contr', 'hours_contr', 'prod']].round(2)

result.to_csv(data_dir / 'gdpjobs.csv', index_label='date')

In [5]:
result

,pop_contr,epop_contr,hours_contr,prod
1989-01-01,NaN,NaN,NaN,NaN
1989-04-01,0.93,-0.31,0.28,2.18
1989-07-01,1.13,0.22,0.24,1.40
1989-10-01,1.09,-0.02,0.03,-0.31
1990-01-01,0.94,-0.68,-0.05,4.23
...,...,...,...,...
2019-04-01,0.43,-0.28,0.50,1.37
2019-07-01,0.57,2.32,0.73,-1.52
2019-10-01,0.56,1.21,0.43,-0.09
2020-01-01,0.39,-2.74,0.00,-2.65
